In [1]:
import pandas as pd

X_train,X_test,y_train,y_test = pd.read_csv('X_train.csv'),pd.read_csv('X_test.csv' ),pd.read_csv('y_train.csv'),pd.read_csv('y_test.csv')

/tmp/ipykernel_34266/3822263571.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
%pip install optuna

  Using cached optuna-3.6.1-py3-none-any.whl.metadata (17 kB)
  Using cached alembic-1.13.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached colorlog-6.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached SQLAlchemy-2.0.29-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached greenlet-3.0.3-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (3.8 kB)
Using cached optuna-3.6.1-py3-none-any.whl (380 kB)
Using cached alembic-1.13.1-py3-none-any.whl (233 kB)
Using cached SQLAlchemy-2.0.29-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
Using cached colorlog-6.8.2-py3-none-any.whl (11 kB)
Using cached greenlet-3.0.3-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (616 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 294.5 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

def objective(trial):
    # Define hyperparameters to be optimized
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    C = trial.suggest_loguniform('C', 0.001, 100)
    
    # Create logistic regression model with specified hyperparameters
    model = LogisticRegression(penalty=penalty, C=C, solver='liblinear', random_state=42)
    
    # Evaluate model using cross-validation
    cv_score = cross_val_score(model, X_train, y_train, cv=5)
    
    # Return negative mean accuracy as Optuna aims to minimize the objective
    return -cv_score.mean()

# Create Optuna study
study = optuna.create_study(direction='minimize')

# Optimize hyperparameters
study.optimize(objective, n_trials=100)

# Get best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", best_params)

# Train logistic regression classifier with best hyperparameters
best_model = LogisticRegression(**best_params, solver='liblinear', random_state=42)
best_model.fit(X_train, y_train)

# Evaluate on test set
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

[I 2024-04-22 11:59:08,108] A new study created in memory with name: no-name-349a4c82-cf47-4bd4-ab23-9ed72726c8ca
/tmp/ipykernel_34266/1458290351.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 100)
[W 2024-04-22 11:59:08,125] Trial 0 failed with parameters: {'penalty': 'l1', 'C': 79.39279657817707} because of the following error: ValueError('Found input variables with inconsistent numbers of samples: [34178, 51003]').
Traceback (most recent call last):
  File "/home/jordigb/.pyenv/versions/3.10.4/envs/PLH/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_34266/1458290351.py", line 15, in objective
    cv_score = cross_val_score(model, X_train, y_train, cv=5)
  File "/home/jordigb/.pyenv/versions/

ValueError: Found input variables with inconsistent numbers of samples: [34178, 51003]